# Open Data Science Conference 2015 (Boston)

# The NYC Taxi Trip Dataset (2013)
### **173 million records**, i.e. taxi trips, 28 GB of uncompressed Data
<img src="nyc_cab.JPG?raw=true" align="left">

http://de.wikipedia.org/wiki/New_York_City_Taxi_Cabs#/media/File:USACab.JPG

# A first look at the data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# TODO: adjust directory
data_dir = '/Users/Felix/taxi_dataset/'

In [ ]:
head = pd.read_csv(data_dir + 'trip_data_1.csv', nrows=20)
head

## qgrid (by Quantopian)

In [ ]:
import qgrid
qgrid.nbinstall()
qgrid.show_grid(head)

# Let's have a second look...this time in Excel

## By default, `pandas` uses
* ### `XlsxWriter` to write `.xlsx`
* ### `openpyxl` to write `.xlsm`
* ### `xlwt` to write `.xls`

In [ ]:
head.to_excel('pandas_out.xlsx', 'Sheet1')

In [ ]:
!open pandas_out.xlsx  # on Windows: !start pandas_out.xlsx

## Manipulating existing Excel sheets is ... limited

* ### Openpyxl can do it, but not everything is supported (e.g. charts are not)

In [ ]:
from openpyxl import load_workbook

writer = pd.ExcelWriter('pandas_out.xlsx', engine='openpyxl')
writer.book = load_workbook('pandas_out.xlsx')
head.to_excel(writer, 'Sheet2')

writer.save()

In [ ]:
!open pandas_out.xlsx  # on Windows: !start pandas_out.xlsx

# `xlwings`: interacts with an open/unsaved Workbook
* ### Windows: by wrapping `pywin32` (COM interface)
* ### Mac: by wrapping `appscript` (AppleScript)
* ### In turn, xlwings needs an installation of Microsoft Excel

In [ ]:
from xlwings import Application, Workbook, Range, Sheet, Chart, ChartType

In [ ]:
wb = Workbook()
Range("A1").value = head
Sheet(1).autofit()

In [ ]:
Sheet.add('Sheet2')
Range('Sheet2', 'B2').value = head
Sheet(2).autofit()

### 2d arrays: nested lists

In [ ]:
# A1 Notation
Sheet(1).activate()
Range('Sheet1', 'H1:I2').value

In [ ]:
# The same with Index notation (Excel-1-based)
Range(1, (1,8),(2,9)).value

In [ ]:
# Get a contiguous Range of cells (as in: "Ctrl-Shift-Down-and-Right")
out = Range(1, 'M2').table.value
out

In [ ]:
# Assign to top-left corner
Range(1, 'B25').value = out

### `table` returns a `Range` object, so we can use any `Range` attribute/method on it:

In [ ]:
Range('B25').table.clear_contents()

### `NumPy` Arrays

In [ ]:
Range(1, 'M2', asarray=True).table.value

### `pandas` DataFrames

In [ ]:
data = Range('B1').table.value
df = pd.DataFrame(data[1:], columns=data[0])
df

# Let's do some `pandas` magic on the Taxi dataset

### `pickup_datetime` and `passenger_count` have been stored in an `HDF5` file (see end of Notebook)

In [ ]:
# No attempts have been made to optimize this...
store = pd.HDFStore(data_dir + 'test.h5')
df = store['df']
store.close()

In [ ]:
df.index

### Let's create a "behavior" table: total passengers per hour and weekday

In [ ]:
df = df.resample('1H', how={'passenger_count': np.sum})
grouped = df.groupby([df.index.time, df.index.weekday])
behavior = grouped['passenger_count'].aggregate(np.sum).unstack()
behavior.columns = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
behavior

### Now let's use Excel to create a heatmap

In [ ]:
wb_heatmap = Workbook()
Range('A1').value = behavior
Range('A:A').number_format = 'HH:MM'

## Let's add a Chart, too

In [ ]:
chart = Chart.add(source_data=Range('A1').table,
                  chart_type=ChartType.xlLine)

### Manipulate attributes

In [ ]:
chart.name

In [ ]:
chart.name = 'taxi'
chart.name

# One more thing: PDF Reporting with ReportLab

In [ ]:
from reportlab.platypus import SimpleDocTemplate, Table

# reportlab initialization with container for Flowables
doc = SimpleDocTemplate("report_basics.pdf")
elements = []

# Create reportlab table from Excel data
data = Range('B1').table.value
table = Table(data)

# Compose content and write PDF document
elements.append(table)
doc.build(elements)

In [ ]:
!open report_basics.pdf

# How to get the data
`$ wget https://nyctaxitrips.blob.core.windows.net/data/trip_data_{1,2,3,4,5,6,7,8,9,10,11,12}.csv.zip`

# Save `passenger_count` into an `HDF5` file

In [ ]:
# Again: not optimized and you'll need a bit of memory to process this...
parts = []
for i in range(1,13):
    part = pd.read_csv(data_dir + 'trip_data_{0}.csv'.format(i),
                       parse_dates=True, index_col='pickup_datetime', 
                       usecols=[5,7], skipinitialspace=True)
    part.sort_index(inplace=True)
    parts.append(part)
    
# Make one big DataFrame and save to HDF5
df = pd.concat(parts)
store = pd.HDFStore(data_dir + 'passenger_count.h5')
store['df'] = df
store.close()

# See also:

http://chriswhong.com/open-data/foil_nyc_taxi/  
http://www.theguardian.com/technology/2014/jun/27/new-york-taxi-details-anonymised-data-researchers-warn  
http://nyctaxi.herokuapp.com/  

See these two Notebooks by Continuum Analytics about how to get the data and how to work with Blaze/bcolz on it:
http://nbviewer.ipython.org/github/ContinuumIO/blaze/blob/gh-pages/notebooks/timings-csv.ipynb
http://nbviewer.ipython.org/github/ContinuumIO/blaze/blob/gh-pages/notebooks/timings-bcolz.ipynb